# GBT 해커톤 경진대회

### 구글드라이브 코렙

구글드라이브의 내 드라이브(MyDrive) 폴더에 GBT해커톤 폴더를 업로드하여 실행하면 됩니다.

- content/drive/MyDrive/GBT해커톤

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 라이브러리 불러오기

In [4]:
import pandas as pd
import numpy as np
import warnings
import re
from tqdm import tqdm
import torch
from types import SimpleNamespace
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from collections import Counter
from transformers import AutoModelForSequenceClassification, AutoTokenizer

### 데이터 전처리

#### 0. 데이터 불러오기

In [5]:
train_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/GBT해커톤/total_code_file/data/train.csv'

#### 1. 영어 번역 부분 제거

기사, 구글 순서일때

In [ ]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

영문, 번역, 오류, 전제 순서일때

In [ ]:
# '키워드' 열에서 '영문', '번역', '오류', '전제' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '영문, 번역, 오류, 전제' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['영문', '번역', '오류', '전제']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('영문')].strip()

# 각 행의 '키워드' 값에서 '영문' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

구글, 번역기, 번역, 영문기사 순서일때

In [ ]:
# '키워드' 열에서 '구글', '번역기', '번역', '영문기사' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '구글, 번역기, 번역, 영문기사' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['구글', '번역기', '번역', '영문기사']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('구글')].strip()

# 각 행의 '키워드' 값에서 '구글' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

구글번역, 번역 순서일때

In [ ]:
# '키워드' 열에서 '구글번역', '번역' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['구글번역', '번역']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['구글번역', '번역']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '구글번역, 번역' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['구글번역', '번역']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('구글번역')].strip()

# 각 행의 '키워드' 값에서 '구글번역' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

기사,Google,번역,번역, 으로 남아있는 문장에 대한 처리

In [ ]:
def remove_last_keywords(text):
    keywords_to_remove = ['기사', 'Google', '번역', '번역']
    words = text.strip(',').split(',')

    # 마지막 4개의 단어가 keywords_to_remove와 일치하는지 확인
    if words[-4:] == keywords_to_remove:
        return ','.join(words[:-4]) + ',' if len(words[:-4]) > 0 else ''

    return text

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_last_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_last_keywords)

한글 -> 영어(7단어이상 이어질때) 처리

In [ ]:
def remove_english_after_korean(text):
    # 한글과 영어 단어를 구분하는 정규 표현식
    korean_pattern = re.compile(r'[가-힣]+')
    english_pattern = re.compile(r'\b[a-zA-Z]+\b')

    english_count = 0
    start_index = 0

    words = text.split(',')

    for i, word in enumerate(words):
        if english_pattern.search(word):
            english_count += 1
            if english_count == 1:
                start_index = i
            if english_count >= 7:
                return ','.join(words[:start_index])
        else:
            english_count = 0  # 영어 단어가 연속되지 않으면 카운트 초기화

    return text

def apply_function_if_last_word_is_english(df, column_name):
    english_pattern = re.compile(r'\b[a-zA-Z]+\b')

    def conditionally_apply(text):
        words = text.split(',')
        if words and english_pattern.search(words[-1]):
            return remove_english_after_korean(text)
        return text

    df[column_name] = df[column_name].apply(conditionally_apply)

# '키워드' 열에 함수 적용
apply_function_if_last_word_is_english(train_df, '키워드')
apply_function_if_last_word_is_english(test_df, '키워드')

html문 제거

In [ ]:
def remove_specific_pattern(text):
    # 시작과 끝 패턴 정의
    start_pattern = r'monica-writing,entry-btn,root'
    end_pattern = r'gc-guide,setting,hover,opacity,0\.8'

    # 시작 패턴이 있는지 확인
    if re.search(start_pattern, text):
        # 끝 패턴이 있는지 확인하고, 있으면 그 부분까지 제거
        if re.search(end_pattern, text):
            # 시작 패턴부터 끝 패턴까지의 부분을 제거
            text = re.sub(f'{start_pattern}.*?{end_pattern},', '', text)

    return text

def apply_pattern_removal(df):
    # 정확히 일치하는 행 필터링
    filtered_df = df[df['키워드'].str.contains(r'monica-writing', na=False)]

    # 필터링된 행에 대해 패턴 제거 함수 적용
    df.loc[filtered_df.index, '키워드'] = filtered_df['키워드'].apply(remove_specific_pattern)

# train_df와 test_df에 함수 적용
apply_pattern_removal(train_df)
apply_pattern_removal(test_df)

#### 2. 중복된 행 및 오타 행 제거

중복된 행 제거

In [ ]:
print(len(train_df))
print(len(test_df))

In [ ]:
train_df.drop_duplicates(subset=['제목', '키워드'], keep='first', inplace=True)

In [ ]:
print(len(train_df))
print(len(test_df))

ᅳᆯ 있는 행 제거

In [ ]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'ᅳᆯ', na=False)]

# 결과 출력
filtered_df.iloc[0]['키워드']

In [ ]:
# 해당 행 제거
train_df = train_df.drop(filtered_df.index)

In [ ]:
print(len(train_df))
print(len(test_df))

#### 3. 사이트 주소 제거

In [ ]:
# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_specific_words)
test_df['키워드'] = test_df['키워드'].apply(remove_specific_words)

In [ ]:
# CSV 파일로 저장
train_df.to_csv('/content/drive/MyDrive/GBT해커톤/total_code_file/data/train_df_ml_data.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('/content/drive/MyDrive/GBT해커톤/total_code_file/data/test_df_ml_data.csv', index=False, encoding='utf-8-sig')

#### 4. 기호 제거

In [ ]:
df_combined = pd.concat([train_df, test_df], axis=0)

In [ ]:
# 기호를 추출하는 함수
def extract_symbols(text):
    return re.findall(r'[^\w\s,]', text)  # 알파벳, 숫자, 공백, 쉼표를 제외한 모든 기호

# '키워드' 열에서 기호 추출
symbols = df_combined['키워드'].apply(extract_symbols)

# 기호 리스트 출력
unique_symbols = set([symbol for sublist in symbols for symbol in sublist])
print(unique_symbols)

In [ ]:
# 제거할 기호 리스트
symbols_to_remove = {'*', '“', '▷', '‧', '■', '○', '!', '⋯', '→', '㎡', ')', '·', '․', '∙', '?',
                     '㈜', '.', '(', '•', ':', '⋅', '%', ';', '/', '˙', '・', '▲', '’', '~'} # '&' '-'

# 기호를 제거하는 함수
def remove_symbols(text):
    pattern = '[' + re.escape(''.join(symbols_to_remove)) + ']'  # 제거할 기호 패턴 생성
    return re.sub(pattern, '', text)

# '키워드' 열에서 기호 제거
train_df['키워드'] = train_df['키워드'].apply(remove_symbols)
test_df['키워드'] = test_df['키워드'].apply(remove_symbols)

#### 5. 영어+한글 불용어 패턴 제거

In [ ]:
# 제거할 단어 패턴 리스트
remove_patterns = [
    r'http', r'www\.', r'\.kr', r'\.net', r'\.com',
    r'[A-Z]씨', r'[A-Z]양', r'[A-Z]군', r'[A-Z]사', r'[A-Z]팀', r'[A-Z]업체', r'[A-Z]노선',
    r'[A-Z]등급', r'[A-Z]자형', r'[A-Z]의원', r'[A-Z]그룹', r'[A-Z]타입', r'[A-Z]교사', r'[A-Z]법인',
    r'[A-Z]선수', r'[A-Z]시행사', r'[A-Z]대표', r'[A-Z]경위', r'[A-Z]시의원', r'[A-Z]센터', r'[A-Z]교회',
    r'[A-Z]아파트', r'[A-Z]학생', r'[A-Z]값', r'[A-Z]공인중개사사무소', r'[A-Z]캐피탈', r'[A-Z]고등학교',
    r'[A-Z]경사', r'[A-Z]농장', r'[A-Z]지구', r'[A-Z]구역', r'[A-Z]산후조리원', r'[A-Z]노래방', r'[A-Z]사령관',
    r'[A-Z]동', r'[A-Z]원장', r'[A-Z]카페', r'[A-Z]연구소', r'[A-Z]주식회사', r'[A-Z]요양병원', r'[A-Z]채널',
    r'[A-Z]연습장', r'[A-Z]학원', r'[A-Z]건설사', r'[A-Z]초등학교', r'[A-Z]시장', r'[A-Z]식품접객업소',
    r'[A-Z]한약국', r'[A-Z]한약방', r'[A-Z]모사', r'[A-Z]회사', r'[A-Z]경찰관', r'[A-Z]기업', r'[A-Z]업체',
    r'[A-Z]공인중개사사무소', r'[A-Z]도시', r'[A-Z]회장', r'[A-Z]동물병원', r'[A-Z]요양원', r'[A-Z]도의원',
    r'[A-Z]변호사', r'[A-Z]마트', r'[A-Z]병원', r'[A-Z]공인중개사', r'[A-Z]협회', r'[A-Z]사장', r'[A-Z]개발',
    r'[A-Z]조합', r'[A-Z]노선', r'[A-Z]구', r'[A-Z]그룹', r'[A-Z]자', r'[A-Z]공', r'[A-Z]검사', r'[A-Z]팀',
    r'[A-Z]중학교', r'[A-Z]건설사', r'[A-Z]컨벤션', r'[A-Z]건설', r'[A-Z]농협', r'[A-Z]약국', r'[A-Z]구역',
    r'[A-Z]은행', r'[A-Z]헬스장', r'[A-Z]팀', r'[A-Z]예식장', r'[A-Z]단지', r'[A-Z]지점', r'[A-Z]지자체',
    r'[A-Z]교회', r'[A-Z]블록', r'[A-Z]목사', r'[A-Z]교사', r'[A-Z]골프장', r'[A-Z]건설', r'[A-Z]종합부동산금융사',
    r'[A-Z]세무서장', r'[A-Z]산업', r'[A-Z]골프장', r'[A-Z]강사', r'[A-Z]원장', r'[A-Z]기업', r'[A-Z]타워',
    r'[A-Z]현장', r'[A-Z]소방관', r'[A-Z]동', r'[A-Z]양', r'[A-Z]초등학생', r'[A-Z]언론사', r'[A-Z]당원',
    r'[A-Z]시', r'[A-Z]상병', r'[A-Z]교수', r'[A-Z]학원', r'[A-Z]아파트', r'[A-Z]등', r'[A-Z]구'
]

# 앞의 알파벳을 삭제하는 함수
def remove_leading_letters(keyword, patterns):
    for pattern in patterns:
        if re.search(pattern, keyword):
            keyword = re.sub(r'^[A-Z]', '', keyword)
    return keyword

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_leading_letters(x, remove_patterns))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_leading_letters(x, remove_patterns))

#### 6. 숫자+한글 불용어 패턴 제거

In [ ]:
# 삭제할 패턴에 포함될 요소 리스트
'''
특정 분류에서 나타날 수 있는 단위와 고유명사와 관련된 글자는 제외
'''
terms = [
    '명', '가정', '가구', '인', '남매', '팀', '건', '권', '개', '편',
    '톤', '대', '곳', '층', '포', '석', '점', '마리', '칸', '기',
    '자', '계단', '도', '호', '주', '채', '평', '표', '물', '살',
    '통', '분', '세', '호실', '여', '회', '종류', '장', '며',
    '야드', '종', '부', '강', '사', '면', '단', '무', '병', '판',
    '남', '쪽', '면', '짜', '절', '종', '사', '정', '길', '노',
    '시', '그릇', '실', '쌍', '차로', '시간', '괘', '교시', '박스',
    '박', '종', '부', '판', '코스', '단지', '구역', '퍼센트',
    '밀리미터', '미터', '그루', '량', '척', '지구', '억', '만',
    '천', '천만', '조', '원', '주년', '년', '월', '일', '초',
    '배', '가량', '차례', '섹션', '번', '길', '구역', '군데', '타입',
    '상자', '나', '베', '매', '갸', '가까이', '백억', '묶음', '킬로톤',
    '달', '리', '밀리', '제곱미터', '리터', '팩', '잔'
]

In [ ]:
# 숫자+한글 삭제 함수 정의
def remove_number_korean(keywords):
    # 정규 표현식 패턴 생성
    pattern = r'\d+\s*(' + '|'.join(terms) + r')'

    # 쉼표로 분리하여 키워드 리스트 생성
    split_keywords = keywords.split(',')

    # 패턴에 맞는 키워드를 제외한 리스트 생성
    filtered_keywords = [kw for kw in split_keywords if not re.search(pattern, kw)]

    return ','.join(filtered_keywords)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_number_korean)
test_df['키워드'] = test_df['키워드'].apply(remove_number_korean)

#### 7. ~씨 제거

In [ ]:
remove_keywords = ['윤씨', '조씨', '이씨', '김씨', '남씨', '홍씨', '김연정씨', '박창수씨', '공씨', '에스씨', '조세형씨', '이정열씨', '황규열김주원씨', '조현증씨', '이해석씨',
                   '임씨', '손선영씨', '손씨', '김영숙씨', '정유민씨', '박혜진씨', '민경업씨', '김만배씨', '김용완씨', '박씨', '황씨', '원씨', '행금씨', '김민숙씨', '원유선씨',
                   '노윤미씨', '김루미씨', '김미영씨', '김미란씨', '이상호씨', '유씨', '엘티씨', '안수진씨', '허희진씨', '박경희씨', '소영씨', '강영옥씨', '은미씨', '천선희씨',
                   '배영수씨', '오창식씨', '조현수씨', '피티씨', '에이피티씨', '양호진씨', '양씨', '오씨', '이세연씨', '인씨', '최씨', '문성윤씨', '배영원씨', '김성애씨', '김진희씨',
                   '주씨', '제로씨', '서씨', '나씨', '권씨', '장씨', '신씨', '이순환씨', '기철씨', '백씨', '강씨', '전씨', '김도혜씨', '장진호씨', '함씨', '주식씨', '송용탁씨',
                   '이보경씨', '이창홍씨', '유나씨', '이광선씨', '윤군선씨', '김환용씨', '정희씨', '한규원씨', '손석민씨', '지영씨', '정의종씨', '김한덕씨', '현정씨', '정한택씨',
                   '태권씨', '김문조씨', '한씨', '박노일씨', '한철수씨', '티이엠씨', '장영자씨', '정씨', '고씨', '모질상씨', '이지학씨', '연씨', '최가영씨', '박유민씨', '정은경씨',
                   '황영희씨', 'ㄷ씨', 'ㄹ씨', '곽선진씨', '곽씨', '안혁종씨', '김준엽씨', '윤여정씨', '임혜영씨', '이대준씨', '이은해씨', '박종근씨', '인민정씨', '윤재영씨',
                   '장순복씨', '백영식씨', '김상호씨', '김명수씨', '나병순씨', '박찬순씨', '이경우씨', '봉씨', '박채윤씨', '정대현씨', '김대규씨', '하청일씨', '남기화씨', '맹기호씨',
                   '맹씨', '오안식씨', '박다혜씨', '송민건씨', '김형식씨', '최혁준씨', '드란티융씨', '명재씨', '최정학씨', '원종씨', '정덕임씨', '한석만씨', '헌욱씨', '김태경씨',
                   '창호씨', '석호씨', '장경수씨', '문씨', '에이치에스씨', '이동건씨', '이창우씨', '이선균씨', '안씨', '김승환씨', '권순찬씨', '정원섭씨', '아용씨', '김우식씨',
                   '박소영씨', '강우석씨', '김지용씨', '우수한씨', '이경모씨', '정지현씨', '엘엠에이씨', '권재찬씨', '김광성씨', '문근욱씨', '김광성ㆍ문근욱씨', '박남숙씨', '박인숙씨',
                   '조현숙씨', '임상혁씨', '김장환씨', '김성도씨', '주상중씨', '김가영씨', '박의진씨', '김숙자씨', '이승현씨', '이석원씨', '최정원씨', '김태완씨', '김태규씨',
                   '김영민씨', '조아라씨', '김영서씨', '서재민씨', '허서진씨', '김윤정씨', '전주최씨', '김용수씨', '김종수씨', '주영순씨', '속헹씨', '손정섭씨', '조성달씨', '시티오씨',
                   '김영남씨', '이동호씨', '선영씨', '전명진씨', '민씨', '김건희씨', '김순자씨', '이아이씨', '박은옥씨', '오정순씨', '피에스엠씨', '김인숙씨', '정현민씨', '김종찬씨',
                   '한경희씨', '배씨', '고제설씨', '오경진씨', '김점연씨', '강금구씨', '케이씨', '남모씨', '이광옥씨', '이상일씨', '남상일씨', '박재성씨', '김민지씨', '이사민씨',
                   '준섭씨', '주무정씨', '장세현씨', '기씨', '석열씨', '황모씨', '송길용씨', '송혜희씨', '송씨', '에프엘씨', '해성티피씨', '설민재씨', '옥경씨', '손병조씨', '김미숙씨',
                   '이용관씨', '박병섭씨', '박만귀씨', '김혜경씨', '서선이씨', '유진식씨', '이정원씨', '류시현씨', '유지연씨', '박철민씨', '김부선씨', '이재선씨', '엘텍유브이씨', '강대하씨',
                   '박준석씨', '박성엽씨', '한은미씨', '김상철씨', '선미영씨', '안재수씨', '김태종씨', '지씨', '김원씨', '오민주씨', '노씨', '장성경씨', '전재욱씨', '박귀현씨',
                   '김경민씨', '이삼영씨', '이숙경씨', '용인이씨', '연안이씨', '한양조씨', '해주오씨', '우봉이씨', '안상준씨', '김영세씨', '최옥희씨', '신영식씨', '원모씨',
                   '윤순영씨강릉김씨', '조동선씨', '인구씨', '시하씨', '미진씨', '최상규씨', '혜랑씨', '한혜경씨', '희선씨', '원일씨', '김학성씨', '김영희씨', '김민구씨', '임찬혁씨',
                   '김혜주씨', '이뮨셀엘씨', '애진씨', '현우진씨', '현씨', '최성현씨', '정해정씨', '김남욱씨', '장남초씨', '고선환씨', '강성돈씨', '최선일씨', '홍선자씨', '이재용씨',
                   '김인섭씨', '장길영씨', '이경애씨', '임한귀씨', '원혜경씨', '김중식씨', '연지호씨', '승일씨', '제남씨', '조민아씨', '인숙씨', '훈국씨', '수진씨', '박기남씨',
                   '원재씨', '조장희씨', '곽세근씨', '김현자씨', '전영기씨', '이명수씨', '박미애씨', '조상현씨', '지모씨', '박용선씨', '안선유씨', '민지영씨', '이윤경씨', '이순재씨',
                   '이에스알씨', '백종원씨', '박성남씨', '김희정씨', '의령남씨', '이상철씨', '은선씨', '진규씨', '성규씨', '지희씨', '상철씨', '최미영씨', '김하은씨', '이시아씨',
                   '이정례씨', '박병달씨', '김은정씨', '조우원씨', '김은아씨', '김용범씨', '김태권씨', '이정숙씨', '임은주씨', '김회묵씨', '김보건씨', '이규환씨', '김주연씨',
                   '김종옥씨', '안미향씨', '남평수씨', '안은태씨', '손기찬씨', '황정미씨', '이수훈씨', '지명규씨', '이주영씨', '류승화씨', '정진우씨', '김금란씨', '허승원씨',
                   '허씨', '송하나씨', '배원형씨', '김주익씨', '박창배씨', '최서현씨', '장은지씨', '조혜진씨', '류씨', '김선자씨', '윤상엽씨', '모니카씨', '이경화씨', '송명옥씨',
                   '박래헌씨', '김대덕씨', '김영복씨', '강천심씨', '이소연씨', '심씨', '영애씨', '안효석씨', '오승훈씨', '정창희씨', '박해옥씨', '이춘식씨', '윤동욱씨', '구지서씨',
                   '상훈씨', '김광록씨', '엔젤리나씨', '이강만씨', '조예림씨', '안선영씨', '디무두씨', '김민영씨', '이영민씨', '김태구씨', '심지훈씨', '대부디케이에이엠씨', '박은출씨',
                   '루스씨', '김예형씨', '남건우씨', '이동원씨', '이성애씨', '배미심씨', '서갑열씨', '오진아씨', '최춘일씨', '이현길씨', '차용용씨', '양윤우씨', '신영준씨',
                   '김옥귀씨', '이영우씨', '이상아씨', '안종일씨', '송창수씨', '진용복씨', '홍라희씨', '조희준씨', '조승제씨', '제보자A씨', '한광수씨', '송윤아씨', '최희수씨',
                   '김대섭씨', '김보라씨', '박세준씨', '김도훈씨', '윤수빈씨', '연합뉴스A씨', '하연이씨', '홍명국씨', '정원영씨', '정경진씨', '김소영씨', '오형식씨', '김성호씨',
                   '김성규씨', '권민혁씨', '진씨', '이봉주씨', '장석동씨', '임옥춘씨', '유병원씨', '이재기씨', '천씨', '김민규씨', '정준영씨', '이정진씨', '정성수씨', '서정애씨',
                   '주광덕씨', '황대한씨', '박승일씨', '이소령씨', '상우씨', '희재씨', '향순씨', '한지은씨', '한영섭씨', '조순임씨', '은정씨', '조규애씨', '오호영씨', '김영재씨',
                   '최은숙씨', '박현식씨', '현식씨', '최수진씨', '문정옥씨', '이현우씨', '김선진씨', '이진우씨', '정태선씨', '김민경씨', '구씨', '박현진씨', '박성준씨', '백건종씨',
                   '이현희씨', '전두환씨', '김미진씨', '김종훈씨', '손석우씨', '현대아이에프씨', '김삼열씨', '안하옥씨', '재오씨', '송진현씨', '조인철씨', '광축씨', '진경씨', '이지혜씨',
                   '오창린씨', '성은씨', '이종성씨', '윤경씨', '이영자씨', '조선애씨', '구제식씨', '홍종모씨', '정수조씨', '제이에스비씨', '박해미씨', '동호씨', '이유진씨', '김성회씨',
                   '김광호씨', '김병지씨', '강성혜씨', '김남희씨', '아로마에스씨', '엔피씨', '강철원씨', '윤도일씨', '박강빈씨', '김정현씨', '김수길씨', '김승현씨', '김시현씨', '변씨',
                   '백송이씨', '안유주씨', '오영정씨', '석지원씨', '석씨', '무무씨', '신선씨', '임대윤씨', '최성진씨', '노은희씨', '노창수씨', '김은철씨', '조봉경씨', '이호재씨', '김선화씨',
                   '김유림씨', '정순옥씨', '김경숙씨', '김은주씨', '김대업씨', '김어준씨', '차씨', '김민하씨', '나은미씨', '장석원씨', '이윤주씨', '조은순씨', '은자영씨', '은씨', '이가영씨',
                   '한재현씨', '김민정씨', '김신영씨', '김길영씨', '김혁준씨', '황은미씨', '정선주씨', '고진혁씨', '한숙영씨', '김진호씨', '김진일씨', '서동숙씨', '최철영씨', '필드날씨', '선기씨',
                   '진영씨', '김명중씨', '한예찬씨', '구슬아씨', '김시은씨', '경주이씨', '전시영씨', '정주희씨', '최민석씨', '주희씨', '안영모씨', '양준혁씨', '김흥태씨', '엘클라씨', '서윤정씨',
                   '김민기씨', '임숙자씨', '이민정씨', '임호진씨', '디씨', '우지원씨', '최수형씨', '김인동씨', '김춘자씨', '오준환씨', '광산김씨', '김서흥씨', '홍길선씨', '버터워스씨', '이재록씨',
                   '이명배씨', '원상희씨', '정지혁씨', '원종남씨', '김도경씨', '김은희씨', '윤해진씨', '김태수씨', '김도식씨', '조정미씨', '최인선씨', '주연씨', '이순자씨', '김창실씨', '김현겸씨',
                   '조주빈씨', '의송씨', '이윤재씨', '신중현씨', '김영수씨', '김민수씨', '최성운씨', '원진호씨', '박현주씨', '이상희씨', '박종국씨', '박기태씨', 'ㅅ씨', '성동욱씨', '노진씨',
                   '김봉진씨', '에스피씨', '박혜숙씨', '유병태씨', '정찬민씨', '남재혁씨', '김장희씨', '김동원씨', '이광재씨', '송정희씨', '백상엽씨', '김병근씨', '김영철씨', '이세영씨', '김창준씨',
                   '임종민씨', '방은주씨', '전순돌씨', '홍남기씨', '강희정씨', '황인상씨', '순관씨', '혜숙씨', '황교익씨', '박수진씨', '김도연씨', '배소영씨', '송중기씨', '이준영씨', '아키라씨',
                   '김상국씨', '김동성씨', '비씨', '신승희씨', '황한기씨', '양전영씨', '김예림씨', '최성일씨', '강철민씨', '김용호씨', '정익희씨', '한경원씨', '양윤선씨', '이종현씨', '최재영씨',
                   '김재우씨', '이동식씨', '요구유씨', '제원씨', '은진씨', '김현기씨', '송갑석씨', '어일선씨', '문길곤씨', '채씨', '케이씨씨', '부인연일정씨', '이덕화씨', '김점도씨', '진광일씨',
                   '이기성씨', '최종영씨', '김진환씨', '마사유키씨', '미즈노씨', '정의찬씨', '임진우씨', '임지숙씨', '박상혁씨', '박명기씨', '정재혁씨', '조중헌씨', '국소정씨', '김도순씨', '임효섭씨',
                   '박정기씨', '최만수씨', '고덕순씨', '이진배씨', '상효씨', '이대우씨', '김성수씨', '김용주씨', '김영순씨', '김상경씨', '심근도씨', '의혹김씨', '양동근씨', '이지현씨', '강재준씨',
                   '옥승호씨', '황준성씨', '조면희씨', '정찬희씨', '임호민씨', '백보람씨', '이동섭씨', 'ㅁ씨', 'ㅂ씨', '희빈장씨', '동원씨', '원진씨', '남재씨', '호순씨', '공명남씨', '김용학씨',
                   '강수철씨', '김동현씨', '임난주씨', '노창옥씨', '이상금씨', '최용수씨', '김지현씨', '손창근씨', '문석씨', '이경환씨', '양기승씨', '규상씨', '기중씨', '김해곤씨', '김해수씨',
                   '황문희씨', '박진우씨', '전해린씨', '아름씨', '박재인씨', '이병윤씨', '신네씨', '산데르씨', '시몬씨', '안네씨', '이종직씨', '한진옥씨', '김은화씨', '우씨', '미숙씨', '조경미씨',
                   '오혜림씨', '김혜림씨', '염씨', '황선아씨', '윤화씨', '엄복준씨', '순영씨', '윤간식씨', '원주씨', '승선씨', '이동준씨', '정규리씨', '도경완씨', '박종석씨', '김병구씨', '이진수씨',
                   '이정린씨', '이원재씨', '박순옥씨', '마씨', '마화영씨', '유인석씨', '안은경씨', '김영경씨', '홍모씨', '임정숙씨', '부스타만테씨', '고영찬씨', '디디씨', '배종영씨', '김수형씨',
                   '김필주씨', '송선혜씨', '김만년씨', '이순길씨', '김인억씨', '조성희씨', '신용호씨', '성경순씨', '허형범씨', '김정묵씨', '이혜경씨', '양승훈씨', '이인옥씨', '한영수씨', '선아씨',
                   '신명씨', '박현철씨', '김대범씨', '박경인씨', '정태인씨', '중전신씨', '이정옥씨', '달평씨', '윤석민씨', '황규열씨', '조근우씨', '정향자씨', '유진씨', '김태수씨', '도식씨',
                   '이영민씨', '이선우씨', '박성은씨', '허씨', '윤재영씨', '장순복씨', '이은해씨', '백숙경씨', '최인숙씨', '민우혁씨', '민씨', '서은광씨', '이순재씨', '정성수씨', '최은숙씨',
                   '도경완씨', '최명상씨', '전씨', '한씨', '신씨', '두연진씨', '최선일씨', '김남욱씨', '김원씨', '김동일씨', '김유라씨', '서씨', '심씨', '전영기씨', '진민경씨', '김한숙씨',
                   '김성범씨', '박종근씨', '김동성씨', '용인이씨', '연안이씨', '한양조씨', '해주오씨', '우봉이씨', '우상전씨', '김만배씨', 'ㄱ씨', '주호민씨', '주씨', '김용주씨', '김영배씨',
                   '박채윤씨', '민관식씨', '안영주씨', '윤성문씨', '원상희씨', '정지혁씨', '백씨', '장씨', '김명수씨', '나병순씨', 'ㄴ씨', '김준형씨', '고씨', '유리아씨', '곽씨', '김신영씨',
                   '정씨', '아로마에스씨', '심지훈씨', '남재형씨', '김영세씨', '대부디케이에이엠씨', '홍모씨', '최혜령씨', '임지현씨', '강희주씨', '강씨', '이선균씨', '김창준씨', '윤순영씨',
                   '임후남씨', '형석씨', '이종욱씨', '허경영씨', '김용학씨', '강수철씨', '김명식씨', '디씨', '박인숙씨', '이수진씨', '윤제남씨', '미숙씨', '이형수씨', '김건희씨', '이명수씨',
                   '송태일씨', '심복선씨', '진중권씨', '하이닉스시스템아이씨', '연씨', '옥승호씨', '안씨', '홍재석씨', '이강만씨', '장영자씨', '드란티융씨', '김상교씨', '최옥희씨', '최수진씨',
                   '문정옥씨', '이현우씨', '김승환씨', '김선진씨', '이진우씨', '정태영씨', '류씨', '엄씨', '권씨', '장길영씨', '이경애씨', '송씨', '박진우씨', '김시은씨', '김기리다씨', '김영희씨',
                   '홍지청씨', '진우씨', '빌레씨', '이아이씨', '신승희씨', '황한기씨', '양전영씨', '김예림씨', '김민규씨', '엄부영씨', '나인선씨', '김명은씨', '이승희씨', '김현중씨', '이태원씨',
                   '오경진씨', '김점연씨', '신두만씨', '캡처A씨', '손정섭씨', '박상혁씨', '박명기씨', '정재혁씨', '조중헌씨', '국소정씨', '김우식씨', '윤호선씨', '유청진씨', '조영호씨', '윤아란씨',
                   '이주원씨', '천씨', '설민재씨', '조은순씨', '권순흥씨', '박기태씨', '최근순씨', '박제임스휘준씨', '김광보씨', '김경숙씨', '김성진씨', '한주식씨', '에이치에스씨', '김한나씨',
                   '이재선씨', '박철민씨', '김부선씨', '김해용씨', '추연창씨', '강지아씨', '양성자씨', '이성희씨', '정명숙씨', '장애희씨', '유동규씨', '정민용씨', '최윤길씨', '호씨', '성상경씨',
                   '임명숙씨', '한아란씨', '박재수씨', '김수연씨', '에프엘씨', '황주연씨', '이순환씨', '성씨', '송주현씨', '박진원씨', '한담희씨', '김상균씨', '나씨', '이영희씨', '이경순씨',
                   '강소현씨', '김순이씨', '곽세근씨', '김영선씨', '한승수씨', '장지호씨', '조덕수씨', '정주원씨', '서원준씨', '유임수씨', '이해성씨', '박용현씨', '김율씨', '박동근씨', '장현진씨',
                   '최상범씨', '정동희씨', '류승호씨', '김민희씨', '최경진씨', '김태경씨', '의령남씨', '안하옥씨', '장기숙씨', '박재우씨', '변씨', '명성에프엠씨', '메리이브씨', '보누씨', 'ㅈ씨',
                   '김순자씨', '정원영씨', '최미영씨', '구지민씨', '구씨', '김자민씨', '한석준씨', '김대호씨', '조여정씨', '김선교씨', '진수민씨', '최서원씨', '문영배씨', '정바울씨', '이경우씨',
                   '황대한씨', '손학규씨', '신영선씨', '이민행씨', '윤여정씨', '박숙현씨', '유재일씨', '디피씨', '김지아씨', '오근환씨', '허승원씨', '송하나씨', '배원형씨', '배씨', '금강이엠씨',
                   '상규씨', '황상모씨', '윤영엽씨', '영록씨', '현씨', '알엑스씨', '현욱씨', '마채언씨', '정경진씨', '정명이씨', '원주씨', '송중기씨', '지씨', '조재윤씨', '박삼종씨', '채씨',
                   '박용근씨', '김인성씨', '박미애씨', '정태인씨', '한정덕솜씨', '김진모씨', '박미숙씨', '김형준씨', '임재경씨', '황정윤씨', '우남식씨', '남대일씨', '김병묵씨', '김신씨', '조준희씨',
                   '박노빈씨', '케이씨씨', '김인억씨', '조성희씨', '지티씨', '최수만씨', '김미향씨', '김영빈씨', '김용균씨', '조동익씨', '민아씨', '이승훈씨', '김수정씨', '문씨', '오승민씨',
                   '정덕임씨', '한석만씨', '손석우씨', '케이알씨', '박영희씨', '노씨', '박선정씨', '윤석분씨', '이완용씨', '김일중씨', '선씨', '장남종씨', '고이준씨', '하씨', '조희원씨', '조재형씨',
                   '조재완씨', '김성길씨', '이삼영씨', '꾸뻬씨', '188512씨', '김흥태씨', '김예지씨', '심복순씨', '이해신씨', '송창호씨', '이정례씨', '곽재근씨', '오준환씨', '김영란씨', '문근욱씨',
                   '김광성씨', '한신애씨', '박찬솔씨', '장병선씨', '최경혜씨', '윤병문씨', '곽선진씨', '엠에스티씨', '전인수씨', '김?모?씨', '한광수씨', '의혹김씨', '김영순씨', '엔엑스씨', '정진욱씨',
                   '박태양씨', '최덕규씨', '박근혜씨', '황예진씨', '전두환씨', '노태우씨', '이정희씨', '전용석씨', '강명희씨', '김미경씨', '신현주씨', '이은희씨', '이영호씨', '강병헌씨', '케이이씨',
                   '혜정씨', '이선경씨', '김희영씨', '윤병철씨', '이희경씨', '박초원씨', '윤지희씨', '김지영씨', '조상현씨', '김혜경씨', '김미숙씨', '이유진씨', '인숙씨', '윤해진씨', '희정씨',
                   '민병철씨', '김영남씨', '최지혜씨', '문종찬씨', '황성원씨', '한영수씨', '김영민씨', '민영주씨', '오후석씨', '박숙경씨', '이승용씨', '박영수씨', '박동진씨', '박정호씨', '마씨',
                   '차인표씨', '김보람씨', '신상민씨', '최상호씨', '정연태씨', '정익희씨', '한경원씨', '양윤선씨', '이종현씨', '최재영씨', '김재우씨', '이동식씨', '김주석씨', '박병후씨', '전영희씨',
                   '프셰므씨', '인민정씨', '김성민씨', '정영씨', '동진씨', '선아씨', '후석씨', '이강민씨', '나모씨', '김가영씨', '윤상엽씨', '김민영씨', '최인식씨', '박철원씨', '박현철씨', '김대범씨',
                   '박경인씨', '추씨', '최완영씨', '박소현씨']


In [ ]:
# 삭제할 키워드를 정규 표현식 패턴으로 변환
pattern = '|'.join(remove_keywords)

# '키워드' 열에서 키워드 삭제
train_df['키워드'] = train_df['키워드'].str.replace(r'(' + pattern + r'),?', '', regex=True)
test_df['키워드'] = test_df['키워드'].str.replace(r'(' + pattern + r'),?', '', regex=True)

In [ ]:
'''
def remove_keywords_from_row(row, keywords_to_remove):
    keywords = row.split(',')
    cleaned_keywords = [kw for kw in keywords if kw not in keywords_to_remove]
    return ','.join(cleaned_keywords)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_keywords_from_row(x, remove_keywords))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_keywords_from_row(x, remove_keywords))
'''

#### 8. 기타 불용어 제거

In [ ]:
# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        #r'[0-9]+[가-힣\u4E00-\u9FFF]+|'      # 숫자+한글
        #r'[가-힣]+[0-9]+|'                   # 한글+숫자
        r'[0-9]+[A-Za-z]+' +                 # 숫자+영어
        r'|[A-Za-z]+[0-9]+' +                # 영어+숫자
        r'|[\u4E00-\u9FFF]+' +               # 한자1
        r'|[\u3400-\u4DBF]+' +               # 한자2
        r'|^[0-9]+$'                         # 숫자만
    )
    valid_keywords = []
    exceptions = ['RE100', 'GS25','3D','G90', 'XM3',
                  'GV70', 'GV80', 'KF94', 'QM6', 'EV6',
                  'HBM', 'IFRS17', 'COP26', 'COP27', 'COP28',
                  'B2B', 'H5N', 'B2C', 'P2P']
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            if word not in exceptions:  # 예외 처리
                removed_keywords.append(word)
            else:
                valid_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ','.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 제거된 단어들 출력
print(f"삭제된 키워드 개수: {len(removed_keywords)}")
print("\n제거된 키워드 20개만 출력: ")
print(removed_keywords[:20])

#### 9. 공통 키워드 제외

In [ ]:
# 많이 등장한 공통 키워드 추출 함수
def filter_keywords(df, min_count=10, common_threshold=50):
    '''
    min_count: 최소 등장 횟수
    common_threshold: 공통 키워드의 최소 빈도수
    ex) min_count=10, common_threshold=50 -> 50개 이상의 분류에서 10번 이상 등장한 키워드만 선택
    '''
    # 키워드 분리 및 집계
    df['키워드'] = df['키워드'].str.split(',')
    df_exploded = df.explode('키워드')  # 각 리스트의 요소를 개별 행으로 나눔

    # 분류별 키워드 집계
    keyword_counts = df_exploded.groupby(['분류', '키워드']).size().reset_index(name='빈도수')

    # n개 이상 등장한 키워드 추출
    filtered_keywords = keyword_counts[keyword_counts['빈도수'] >= min_count]

    # 공통 키워드 찾기
    keyword_occurrences = filtered_keywords['키워드'].value_counts()
    common_keywords = keyword_occurrences[keyword_occurrences >= common_threshold].index.tolist()

    # 삭제되는 키워드와 개수
    num_deleted_keywords = len(common_keywords)

    # 삭제되는 키워드의 총 등장 횟수 계산
    total_deleted_occurrences = 0
    for kw in common_keywords:
        total_deleted_occurrences += keyword_occurrences.get(kw, 0)

    # 결과 출력
    print("삭제되는 키워드:", common_keywords)
    print("삭제되는 키워드의 개수(unique): {}".format(num_deleted_keywords))
    print("삭제되는 키워드의 총 개수: {}".format(total_deleted_occurrences))

In [ ]:
df = train_df.copy()
filter_keywords(df, 15, 45)

In [ ]:
# 공통 키워드를 제거하는 함수
def remove_common_keywords(df, common_keywords):
    for index, row in df.iterrows():
        keywords = row['키워드'].split(',')
        # 공통 키워드를 제외한 새로운 키워드 리스트
        filtered_keywords = [kw for kw in keywords if kw not in common_keywords]
        # 필터링된 키워드를 문자열로 다시 합침
        df.at[index, '키워드'] = ','.join(filtered_keywords)
    return df

In [ ]:
common_keywords_list = ['용인시', '경기', '서울', '경기도', '상황', '진행', '대표', '용인'
                        , '이날', '지역', '시작', '예정', '관계자', '생각', '사진', '설명'
                        , '참여', '시장', '코로나19', '활동', '마련', '지원', '전국', '운영'
                        , '시민', '제공', '한국', '시민들', '준비', '모습', '가능', '노력'
                        , '사람', '계획', '정도', '자리', '기록', '확인', '대상', '위치']

train_df = remove_common_keywords(train_df, common_keywords_list)
test_df = remove_common_keywords(test_df, common_keywords_list)

#### 10. 키워드 열 정리

In [ ]:
train_df.head()

In [ ]:
# 키워드를 정리하는 함수
def clean_keywords(keywords):
    # 쉼표로 분리한 후 양쪽 공백 제거 및 빈 키워드 제외
    cleaned_keywords = [keyword.strip() for keyword in keywords.split(',') if keyword.strip()]
    # 공백으로 결합하여 하나의 문자열로 만듦
    return ' '.join(cleaned_keywords)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(clean_keywords)
test_df['키워드'] = test_df['키워드'].apply(clean_keywords)

In [ ]:
# CSV 파일로 저장
train_df.to_csv('/content/drive/MyDrive/GBT해커톤/total_code_file/data/train_df_1012.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('/content/drive/MyDrive/GBT해커톤/total_code_file/data/test_df_1012.csv', index=False, encoding='utf-8-sig')

#### 11. csv 저장

### 모델링

#### DL : roterta

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/train_df_1012.csv")
test_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/test_df_1012.csv")

In [ ]:
config = {
    "learning_rate": 5e-5,
    "epoch": 9,
    "batch_size": 32
}

CFG = SimpleNamespace(**config)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 모델 1: klue/roberta-base 로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
roberta_model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=len(train_df['분류'].unique()))
roberta_model.load_state_dict(torch.load('/content/drive/MyDrive/GBT해커톤/total_code_file/model/roberta.pth'))
roberta_model.to(device)
roberta_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-39-87cc3052f0d2>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly al

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256): # 클래스 초기화 (__init__ 메서드)
        self.texts = texts                                      # texts: 텍스트 데이터의 리스트입니다.
        self.labels = labels                                    # labels: 각 텍스트에 대한 레이블(정답)의 리스트입니다. 레이블이 없는 경우를 위해 기본값을 None으로 설정할 수 있습니다.
        self.tokenizer = tokenizer                              # tokenizer: 텍스트를 토큰화하는 데 사용할 토크나이저 객체입니다. 주로 BERT와 같은 모델의 토크나이저를 사용합니다.
        self.max_len = max_len                                  # max_len: 토큰화된 텍스트의 최대 길이입니다. 기본값은 128로 설정되어 있습니다.

    def __len__(self):                                          # 데이터셋 길이 (__len__ 메서드)
        return len(self.texts)                                  # 데이터셋의 크기를 반환합니다. 즉, 텍스트의 개수를 반환합니다.

    def __getitem__(self, item):                                # 데이터 항목 가져오기 (__getitem__ 메서드)
        text = str(self.texts[item])                            # item: 데이터셋에서 가져올 특정 인덱스입니다.
        label = self.labels[item] if self.labels is not None else -1 # label: 해당 인덱스의 레이블을 가져옵니다. 레이블이 없으면 -1로 설정합니다.
        encoding = self.tokenizer.encode_plus(                  # encoding: 토크나이저의 encode_plus 메서드를 사용하여 텍스트를 인코딩합니다. 이 과정에서 다음과 같은 옵션을 설정합니다:
            text,                                              #
            add_special_tokens=True,                            # add_special_tokens=True: 모델에 필요한 특별한 토큰을 추가합니다.
            max_length=self.max_len,                            # max_length=self.max_len: 최대 길이를 설정합니다.
            return_token_type_ids=False,                        #
            padding='max_length',                               # padding='max_length': 최대 길이에 맞춰 패딩을 추가합니다.
            truncation=True,                                    # truncation=True: 최대 길이를 초과하는 경우 잘라냅니다.
            return_attention_mask=True,                         # return_attention_mask=True: 어텐션 마스크도 반환합니다.
            return_tensors='pt',                                # return_tensors='pt': PyTorch 텐서 형식으로 반환합니다.
        )
        return {                                                # 반환하는 딕셔너리에는 원본 텍스트, 인코딩된 입력 ID, 어텐션 마스크, 레이블이 포함
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)   # train_df에서 키워드와 label을 리스트로 변환하여 TextDataset 클래스의 인스턴스를 생성
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)         # val_df에서 키워드와 label을 사용하여 검증 데이터셋을 생성
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)                        # test_df에서 키워드를 리스트로 변환하여 TextDataset을 생성합니다. 이 데이터셋은 레이블이 없으므로 None을 전달

# 데이터 로더 생성, DataLoader는 데이터를 미니 배치 단위로 나누어서 제공해주는 역할
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [ ]:
# 테스트 세트 데이터셋 및 데이터로더 생성
test_dataset = TextDataset(test_df['키워드'].tolist(), None, tokenizer)  # 테스트 데이터셋 생성 (레이블은 None)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

# 테스트 세트에 대한 예측 수행
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = roberta_model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]


Testing: 100%|██████████| 732/732 [05:32<00:00,  2.20it/s]


In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/sample_submission.csv")
submission["분류"] = decoded_predictions

submission.to_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/roberta_pred.csv", encoding='UTF-8-sig', index=False)

#### DL : kobigbird

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/train_df_1012.csv")
test_df = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/test_df_1012.csv")

In [ ]:
config = {
    "learning_rate": 5e-5,
    "epoch": 10,
    "batch_size": 32
}

CFG = SimpleNamespace(**config)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 모델 2: kobigbird 로드
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/GBT해커톤/total_code_file/model/kobigbird_model")
# 모델 로드
kobigbird_model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/GBT해커톤/total_code_file/model/kobigbird_model",
    num_labels=len(train_df['분류'].unique())  # 분류 라벨 수 설정
)

# 모델을 GPU로 이동
kobigbird_model.to(device)

# 평가 모드로 설정
kobigbird_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256): # 클래스 초기화 (__init__ 메서드)
        self.texts = texts                                      # texts: 텍스트 데이터의 리스트입니다.
        self.labels = labels                                    # labels: 각 텍스트에 대한 레이블(정답)의 리스트입니다. 레이블이 없는 경우를 위해 기본값을 None으로 설정할 수 있습니다.
        self.tokenizer = tokenizer                              # tokenizer: 텍스트를 토큰화하는 데 사용할 토크나이저 객체입니다. 주로 BERT와 같은 모델의 토크나이저를 사용합니다.
        self.max_len = max_len                                  # max_len: 토큰화된 텍스트의 최대 길이입니다. 기본값은 128로 설정되어 있습니다.

    def __len__(self):                                          # 데이터셋 길이 (__len__ 메서드)
        return len(self.texts)                                  # 데이터셋의 크기를 반환합니다. 즉, 텍스트의 개수를 반환합니다.

    def __getitem__(self, item):                                # 데이터 항목 가져오기 (__getitem__ 메서드)
        text = str(self.texts[item])                            # item: 데이터셋에서 가져올 특정 인덱스입니다.
        label = self.labels[item] if self.labels is not None else -1 # label: 해당 인덱스의 레이블을 가져옵니다. 레이블이 없으면 -1로 설정합니다.
        encoding = self.tokenizer.encode_plus(                  # encoding: 토크나이저의 encode_plus 메서드를 사용하여 텍스트를 인코딩합니다. 이 과정에서 다음과 같은 옵션을 설정합니다:
            text,                                              #
            add_special_tokens=True,                            # add_special_tokens=True: 모델에 필요한 특별한 토큰을 추가합니다.
            max_length=self.max_len,                            # max_length=self.max_len: 최대 길이를 설정합니다.
            return_token_type_ids=False,                        #
            padding='max_length',                               # padding='max_length': 최대 길이에 맞춰 패딩을 추가합니다.
            truncation=True,                                    # truncation=True: 최대 길이를 초과하는 경우 잘라냅니다.
            return_attention_mask=True,                         # return_attention_mask=True: 어텐션 마스크도 반환합니다.
            return_tensors='pt',                                # return_tensors='pt': PyTorch 텐서 형식으로 반환합니다.
        )
        return {                                                # 반환하는 딕셔너리에는 원본 텍스트, 인코딩된 입력 ID, 어텐션 마스크, 레이블이 포함
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)   # train_df에서 키워드와 label을 리스트로 변환하여 TextDataset 클래스의 인스턴스를 생성
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)         # val_df에서 키워드와 label을 사용하여 검증 데이터셋을 생성
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)                        # test_df에서 키워드를 리스트로 변환하여 TextDataset을 생성합니다. 이 데이터셋은 레이블이 없으므로 None을 전달

# 데이터 로더 생성, DataLoader는 데이터를 미니 배치 단위로 나누어서 제공해주는 역할
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [ ]:
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = kobigbird_model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 732/732 [06:19<00:00,  1.93it/s]


In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/sample_submission.csv")
submission["분류"] = decoded_predictions

submission.to_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/kobigbird_pred.csv", encoding='UTF-8-sig', index=False)

#### ML : logistic regression

In [6]:
import pandas as pd
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# ----- 1. 데이터 불러오기 ----- #

RANDOM_STATE = 110

# 경고 메시지 억제
warnings.filterwarnings('ignore')

# ml 모델용 데이터셋 사용
df_train = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/train_df_ml_data.csv")
df_test = pd.read_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/test_df_ml_data.csv")

# ----- 2. 모델링(학습) ----- #

# 텍스트와 레이블 분리
X_train = df_train['키워드']  # 키워드 컬럼
y_train = df_train['분류']  # 카테고리 컬럼
X_test = df_test['키워드']

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Count Vectorization
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# 로지스틱 회귀 모델 정의
model_count = LogisticRegression(random_state=RANDOM_STATE)

# 모델 학습
model_count.fit(X_train_count, y_train_encoded)

# ----- 3. 모델링(예측) ----- #
# 예측
count_predictions = model_count.predict(X_test_count)

# 예측 결과를 데이터프레임으로 변환
count_results = pd.DataFrame({'ID': df_test['ID'], '분류': label_encoder.inverse_transform(count_predictions)})

# 결과를 CSV 파일로 저장
count_results.to_csv("/content/drive/MyDrive/GBT해커톤/total_code_file/data/lr_pred.csv", index=False, encoding='utf-8-sig')

#### voting

In [ ]:
def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)

    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]

    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)

    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")

    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

lr, roberta, kobigbird 3개의 모델에 대해서 voing 진행  
이때 3개의 모델에서 다수표가 존재하지 않는 경우  
lr의 타겟(분류)값을 최종 예측값으로 저장  

In [ ]:
# 공통 경로
common_path = "/content/drive/MyDrive/GBT해커톤/total_code_file/data/"

# 제출 파일 이름 리스트
file_names = [
    "lr_pred.csv"
    , "roberta_pred.csv"
    , "kobigbird_pred.csv"
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 제출 파일에서 예측 결과 읽어오기
def read_submission_files(file_paths):
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        if 'ID' not in df.columns or '분류' not in df.columns:
            raise KeyError(f"'ID' 또는 '분류' 열이 {file_path} 파일에 없습니다. 열 이름: {df.columns.tolist()}")
        predictions.append(df[['ID', '분류']])
    return predictions

# 하드 보팅 함수
def hard_voting(predictions):
    final_predictions = {}
    id_list = predictions[0]['ID'].tolist()

    for id in id_list:
        votes = [df[df['ID'] == id]['분류'].values[0] for df in predictions]
        vote_counts = Counter(votes)
        if vote_counts.most_common(1)[0][1] > 1:
            final_predictions[id] = vote_counts.most_common(1)[0][0]
        else:
            final_predictions[id] = votes[0]

    return final_predictions

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "/content/drive/MyDrive/GBT해커톤/total_code_file/" + "total_code_submission.csv"

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub['분류'] = df_sub['ID'].apply(lambda x: final_predictions_hard[x])
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

최종 제출 파일이 '/content/drive/MyDrive/GBT해커톤/total_code_file/total_code_submission.csv'로 저장되었습니다.


### 최종 제출 파일 확인

In [ ]:
import pandas as pd

# 파일 읽기
df_result = pd.read_csv(output_file_name)

# 내용 확인
print(df_result['분류'].value_counts())

분류
지역               11828
경제:부동산            1482
사회:사건_사고          1159
경제:반도체             990
사회:사회일반            545
사회:의료_건강           457
사회:교육_시험           452
정치:국회_정당           410
스포츠:올림픽_아시안게임      359
경제:취업_창업           358
스포츠:골프             282
경제:산업_기업           281
문화:전시_공연           277
경제:자동차             276
정치:선거              266
사회:장애인             259
경제:유통              253
경제:경제일반            229
IT_과학:모바일          219
사회:노동_복지           195
사회:여성              194
경제:서비스_쇼핑          181
경제:무역              162
사회:환경              159
경제:금융_재테크          140
스포츠:축구             138
문화:방송_연예           137
정치:행정_자치           133
국제                 123
정치:청와대             119
문화:출판              101
IT_과학:IT_과학일반       95
IT_과학:과학            95
IT_과학:인터넷_SNS       93
문화:미술_건축            90
문화:학술_문화재           87
문화:요리_여행            77
경제:자원               75
문화:문화일반             73
문화:종교               72
정치:정치일반             66
IT_과학:콘텐츠           57
사회:날씨               54
스포츠:농구_배